In [1]:
import pandas as pd
import string
import numpy as np

In [2]:
df=pd.read_csv("test.csv",encoding="ISO-8859-1")
df=df[['text','sentiment']]
df=df.dropna()


In [3]:
df.text=df.text.str.lower().str.translate(str.maketrans('', '', string.punctuation))


In [4]:
df

,text,sentiment
0,last session of the day httptwitpiccom67ezh,neutral
1,shanghai is also really exciting precisely s...,positive
2,recession hit veronique branquinho she has to ...,negative
3,happy bday,positive
4,httptwitpiccom4w75p i like it,positive
...,...,...
3529,its at 3 am im very tired but i cant sleep bu...,negative
3530,all alone in this old house again thanks for ...,positive
3531,i know what you mean my little dog is sinking...,negative
3532,sutra what is your next youtube video gonna be...,positive


In [5]:
df=df.sample(frac=1)


In [6]:
for i in df.loc[df.sentiment[df.sentiment=='positive'].iloc[:5].index, 'text']:
    print(i)
    print("----------")


 httptwitpiccom4l85a  best kind of gum ever i just bought a pack
----------
i need a present for my mom  could anybody help me 
----------
thank god for some overcast and ivory trying to get my mom to take me out to lunch egh doesnt look like its gonna work
----------
leme put this bottle of ciroc in yababy i can make you popular
----------
 aww you are so helpful  we getting sweepy here sis says we can have whipped cream before bed  i save u some
----------


In [7]:
for i in df.loc[df.sentiment[df.sentiment=='negative'].iloc[:5].index, 'text']:
    print(i)
    print("----------")


 im sorry
----------
omg  madness just came on the radio
----------
recession hit veronique branquinho she has to quit her company such a shame
----------
k2k  y360 closing get out
----------
jus got back from a run up sunset blvd my cuzin tried to kill me my legs are still movin and im sittin down
----------


In [8]:
for i in df.loc[df.sentiment[df.sentiment=='neutral'].iloc[:5].index, 'text']:
    print(i)
    print("----------")


 twappy httpbitlyd9qvk
----------
is not ready for school yet  httpplurkcompwy69d
----------
 or ftrh  for the ride home
----------
first surprise birthday ever best night ive had in a  long time
----------
capones island in 2 days my summer is just starting
----------


In [9]:
training_df=df.iloc[:int(len(df)*0.7)]
testing_df=df.iloc[int(len(df)*0.7):]


In [10]:
training_df

,text,sentiment
3191,twappy httpbitlyd9qvk,neutral
1555,im sorry,negative
1047,httptwitpiccom4l85a best kind of gum ever i ...,positive
603,i need a present for my mom could anybody hel...,positive
2473,is not ready for school yet httpplurkcompwy69d,neutral
...,...,...
1689,guess who has internet at her house raises han...,neutral
1417,no kickboxing classes today pretty bummed sti...,negative
1502,hearing that my songs brighten someones day a...,positive
1081,very sweet haha i am like super proud to be ...,positive


In [11]:
testing_df

,text,sentiment
3144,really annoyed that work appear to have blocke...,negative
1234,ahh i love chinese music haha not gonna see m...,positive
2628,my cankle is sore,negative
2390,oh mann me likey that but sadly im not bein a...,negative
3151,you writing fanfic me read,neutral
...,...,...
1933,at the concert i hope makes a guest appearance,neutral
244,my sister in law just left for her prom she lo...,positive
362,i have nothing witty to say,neutral
551,3 your gonna be the first twitter cause you...,positive


In [12]:
fraction_neutral = (training_df['sentiment']=='neutral').mean()
fraction_positive=( training_df['sentiment']=='positive').mean()
fraction_negative= (testing_df['sentiment']=='negative').mean()


In [13]:
train_positive_words=''.join(training_df[training_df['sentiment']=='positive'].text).split(' ')
train_negative_words=''.join(training_df[training_df['sentiment']=='negative'].text).split(' ')
train_neutral_words=''.join(training_df[training_df['sentiment']=='neutral'].text).split(' ')



In [14]:
common_words_pn=set(train_positive_words).intersection(set(train_negative_words))
common_words=set(common_words_pn).intersection(set(train_neutral_words))

In [15]:
train_neutral_bow=dict()
for w in common_words:
    train_neutral_bow[w]=train_neutral_words.count(w)/len(train_neutral_words)
    
train_positive_bow=dict()
for w in common_words:
    train_positive_bow[w]=train_positive_words.count(w)/len(train_positive_words)
train_negative_bow=dict()
for w in common_words:
    train_negative_bow[w]=train_negative_words.count(w)/len(train_negative_words)


In [16]:
for key, value in list(train_positive_bow.items())[:10]:
    print(f'{key}: {value}')

: 0.055783265020493855
definitely: 0.0006997900629811056
always: 0.000999700089973008
need: 0.0013995801259622112
are: 0.0038988303508947315
over: 0.000499850044986504
3d: 9.997000899730081e-05
believe: 0.00029991002699190244
youre: 0.001599520143956813
dishes: 9.997000899730081e-05


In [17]:
for key, value in list(train_negative_bow.items())[:10]:
    print(f'{key}: {value}')

: 0.045247395833333336
definitely: 0.00021701388888888888
always: 0.0008680555555555555
need: 0.0013020833333333333
are: 0.003689236111111111
over: 0.001736111111111111
3d: 0.00021701388888888888
believe: 0.0003255208333333333
youre: 0.0007595486111111111
dishes: 0.00010850694444444444


# Classification

Predict on Test Set P(positive|"awww holly shes so cute")∝ P("awww holly shes so cute "|Positive)*P(positive)

we make IID assumption on the words so it can be written as

P("aww"|positive).P("holly"|positive)...................P("cute"|postive)*P(postive)




# Predicition

In [18]:
t='she is so cute'
valid=(w for w in t if w in train_positive_words)
print(valid)

<generator object <genexpr> at 0x00000265A3578FB0>


In [19]:
def predict(t,verbose=False):
    valid_words = [w for w in t.split() if w in train_positive_bow]

    positive_prob=[train_positive_bow[w] for w in valid_words]
    negative_prob=[train_negative_bow[w] for w in valid_words]
    neutral_prob=[train_neutral_bow[w] for w in valid_words]

    
    if verbose:
        df=pd.DataFrame()
        df['words']=valid_words
        df['positive_prob']=positive_prob
        df['negative_prob']=negative_prob
        df['neutral_prob']=neutral_prob

        df['ratio_1']=[s/n if n>0 else np.inf for s,n in zip(positive_prob,negative_prob)]
        df['ratio_2']=[s/n if n>0 else np.inf for s,n in zip(positive_prob,neutral_prob)]
        df['ratio_3']=[s/n if n>0 else np.inf for s,n in zip(negative_prob,neutral_prob)]
                
        print(df)
    
    pos_score=sum([np.log(p) for p in positive_prob])+np.log(fraction_positive)
    
    neg_score=sum([np.log(p) for p in negative_prob])+np.log(fraction_negative)

    neutral_score=sum([np.log(p) for p in neutral_prob])+np.log(fraction_neutral)

    if verbose:
        print("positive score is :",pos_score)
        print("Negative score is :",neg_score)
        print('neutral Score is :',neutral_score)
    if pos_score>=neg_score and pos_score>=neutral_score:
        return "positive"
    if neg_score>=pos_score and neg_score>=neutral_score:
        return "negative"
    if neutral_score>=neg_score and pos_score<=neutral_score:
        return "neutral"

In [20]:
predict("she is not so  cute",True)

  words  positive_prob  negative_prob  neutral_prob   ratio_1   ratio_2  \
0   she       0.001999       0.001194      0.000985  1.675134  2.030391   
1    is       0.009097       0.010525      0.009191  0.864335  0.989816   
2   not       0.002899       0.005100      0.006811  0.568476  0.425648   
3    so       0.008198       0.008030      0.005908  1.020926  1.387434   
4  cute       0.000500       0.000109      0.000328  4.606618  1.522793   

    ratio_3  
0  1.212077  
1  1.145176  
2  0.748753  
3  1.358995  
4  0.330566  
positive score is : -30.30811588427425
Negative score is : -31.779847632420164
neutral Score is : -30.675554171375886


'positive'

# The above is not correctly classified cause we have taken IID assumption so although there is not present in the sentence but since the effect of positive prob of other words is greater which makes the total prob positive

In [21]:
predict("she is  so  cute")

'positive'

In [22]:
predict("lets meet, its gonna be crazy")

'positive'

In [23]:
predictions=testing_df.text.apply(lambda t: predict(t))
predictions

3144     neutral
1234    positive
2628    negative
2390    negative
3151     neutral
          ...   
1933    negative
244      neutral
362     negative
551      neutral
2745     neutral
Name: text, Length: 1061, dtype: object

In [24]:
predictions=testing_df.text.apply(lambda t: predict(t))
print(np.sum(testing_df['sentiment']=='positive'))
t=np.sum((predictions == 'positive') & (testing_df['sentiment'] == 'positive'))
print(t)
recall = np.sum((predictions == 'positive') & (testing_df['sentiment'] == 'positive')) / np.sum(testing_df['sentiment'] == 'positive')
print('Fraction positive Correctly Detected: %s'%recall)


316
192
Fraction positive Correctly Detected: 0.6075949367088608


In [25]:
# Assuming predictions and true_sentiments are Pandas Series or NumPy arrays
recall_positive = np.sum((predictions == 'positive') & (testing_df['sentiment'] == 'positive')) / np.sum(testing_df['sentiment']== 'positive')
recall_negative = np.sum((predictions == 'negative') & (testing_df['sentiment']=='negative')) / np.sum(testing_df['sentiment'] == 'negative')
recall_neutral = np.sum((predictions == 'neutral') & (testing_df['sentiment']== 'neutral')) / np.sum(testing_df['sentiment']== 'neutral')

print('Recall for Positive Sentiment:', recall_positive)
print('Recall for Negative Sentiment:', recall_negative)
print('Recall for Neutral Sentiment:', recall_neutral)


Recall for Positive Sentiment: 0.6075949367088608
Recall for Negative Sentiment: 0.48
Recall for Neutral Sentiment: 0.5213483146067416
